# About Mask Language Model - Task 1 - QP



### Library and Parameter

In [ ]:
# Importing necessary libraries and modules for natural language processing and machine learning tasks
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import transformers
from transformers import TrainingArguments, Trainer, pipeline
from transformers import RobertaForMaskedLM, RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import AdamW
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 6
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05

## Classification QP

In [ ]:
!pip install RUST

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 580.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 29.5 MB/s eta 0:00:00


In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# Load data from google drive
from google.colab import drive
drive.mount('/content/drive')

# Load and preprocess training dataset from the JSON file
with open('drive/MyDrive/1.DATA/QP/Numeracy600K_headline_train.json', 'r') as file:
    training_data = json.load(file)
    training_df = pd.DataFrame(training_data)

# Load and preprocess validation dataset from another JSON file
with open('drive/MyDrive/1.DATA/QP/Numeracy600K_headline_test.json', 'r') as file:
    validation_data = json.load(file)
    validation_df = pd.DataFrame(validation_data)

In [ ]:
# Checking unique values in the 'magnitude' column of the training dataframe
training_df['magnitude'].unique()

# Displaying descriptive statistics to analyze the distribution
training_df.describe()

# Inspecting the values of the first row in the training dataframe for initial data exploration
training_df.iloc[0]

In [ ]:
class MagnitudeData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        """
        Custom PyTorch Dataset class for processing data with text and magnitude labels.

        Args:
        - dataframe: Pandas DataFrame containing 'title' (text data) and 'magnitude' columns
        - tokenizer: Tokenizer object for encoding text inputs
        - max_len: Maximum length of tokenized sequences

        Initializes the dataset with necessary attributes.
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = self.data.title
        self.targets = self.data.magnitude
        self.max_len = max_len

    def __len__(self):
        """
        Defines the length of the dataset.

        Returns:
        - Length of the text data
        """
        return len(self.text)

    def __getitem__(self, index):
        """
        Retrieves a single data sample from the dataset.

        Args:
        - index: Index to retrieve the data sample

        Returns:
        - Dictionary containing tokenized inputs and targets as PyTorch tensors
        """
        text = str(self.text[index])
        text = " ".join(text.split())

        # Tokenizing the text input
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        # Constructing and returning the processed data sample
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Splits the dataset into training and testing sets based on a specified ratio,
# then creates instances of the MagnitudeData class for both sets using a tokenizer and maximum length.

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

train_size = 0.3
train_data=training_df.sample(frac=train_size,random_state=200)
test_data=training_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(training_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MagnitudeData(train_data, tokenizer, MAX_LEN)
testing_set = MagnitudeData(test_data, tokenizer, MAX_LEN)

In [ ]:
# Configures data loaders for training and testing using DataLoader from PyTorch,
# with specified batch sizes and settings for shuffling and workers.

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
from torch.utils.data import Dataset, DataLoader

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self, model):
        """
        Initializes the RobertaClass module.

        Args:
        - model: Pre-trained RoBERTa model

        Initializes the layers and components needed for the custom classification model.
        """
        super(RobertaClass, self).__init__()
        self.l1 = model  # Initializing the base pre-trained RoBERTa model
        self.pre_classifier = torch.nn.Linear(768, 768)  # Adding a linear layer for feature transformation
        self.dropout = torch.nn.Dropout(0.3)  # Adding dropout layer for regularization
        self.classifier = torch.nn.Linear(768, 8)  # Adding a linear layer for final classification

    def forward(self, input_ids, attention_mask, token_type_ids):
        """
        Defines the forward pass through the model architecture.

        Args:
        - input_ids: Tokenized input IDs
        - attention_mask: Attention mask for handling padding tokens
        - token_type_ids: Token type IDs for sequence classification

        Returns:
        - Output logits for classification
        """
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]  # Extracts the hidden state from RoBERTa's output
        pooler = hidden_state[:, 0]  # Extracts the pooled output (CLS token)
        pooler = self.pre_classifier(pooler)  # Passes the pooled output through the linear layer
        pooler = torch.nn.ReLU()(pooler)  # Applies ReLU activation function
        pooler = self.dropout(pooler)  # Applies dropout for regularization
        output = self.classifier(pooler)  # Final classification through linear layer
        return output

In [ ]:
# Add new mask for [Num]

# Define a dictionary containing a special token '[Num]' to be added as a mask token
special_tokens_dict = {'mask_token': '[Num]'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
roberta = RobertaModel.from_pretrained("roberta-base")

# Resize the token embeddings in the RoBERTa model to accommodate the added tokens
roberta.resize_token_embeddings(len(tokenizer))
model = RobertaClass(roberta)
model.to(device)

# Print the newly added mask token (should display '[Num]')
print(tokenizer.mask_token)

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on 80% of the dataset to tune the DistilBERT model
def train(epoch):
    tr_loss = 0  # Initializing training loss
    n_correct = 0  # Initializing the number of correct predictions
    nb_tr_steps = 0  # Counting the number of training steps
    nb_tr_examples = 0  # Counting the number of training examples
    model.train()  # Setting the model to train mode

    # Iterating over the training_loader (data loader containing training samples)
    for _, data in tqdm(enumerate(training_loader, 0)):
        # Extracting inputs and targets from the training data
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.long)

        # Forward pass through the model to obtain predictions
        outputs = model(ids, mask, token_type_ids)

        # Calculating the loss between predicted outputs and actual targets
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()  # Accumulating the total training loss

        # Calculating accuracy for the batch
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1  # Incrementing the number of training steps
        nb_tr_examples += targets.size(0)  # Accumulating the total number of training examples processed

        # Displaying training metrics at certain intervals (every 5000 steps in this case)
        if _ % 5000 == 0:
            loss_step = tr_loss / nb_tr_steps
            accu_step = (n_correct * 100) / nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()  # Zeroing out gradients to avoid accumulation
        loss.backward()  # Backpropagation: computing gradients
        optimizer.step()  # Optimizer step: updating weights based on gradients

        # Breaking the loop after a certain number of steps (for demonstration purposes)
        if _ == 15000:
            break

    # Displaying overall accuracy and loss for the epoch
    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct * 100) / nb_tr_examples}')
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return  # Returning from the function

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)

In [ ]:
def valid(model, testing_loader):
    model.eval()  # Sets the model to evaluation mode
    n_correct = 0
    n_wrong = 0
    total = 0
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0

    # Disables gradient calculation for evaluation
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)

            # Obtains model predictions for the input data
            outputs = model(ids, mask, token_type_ids).squeeze()

            # Calculates the loss between predictions and actual targets
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()  # Accumulating validation loss

            # Calculating accuracy for the batch
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            # Displaying validation metrics at certain intervals (every 5000 steps in this case)
            if _ % 5000 == 0:
                loss_step = tr_loss / nb_tr_steps
                accu_step = (n_correct * 100) / nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
                break  # Breaking the loop for demonstration

    # Calculating and displaying overall validation accuracy and loss for the epoch
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu  # Returning the epoch-level accuracy for validation

In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

In [ ]:
output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

## Inference QP

In [ ]:
# Load data from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load and preprocess training dataset from the JSON file
with open('drive/MyDrive/1.DATA/QP/Numeracy600K_headline_train.json', 'r') as file:
    training_data = json.load(file)
    training_df = pd.DataFrame(training_data)

# Load and preprocess validation dataset from another JSON file
with open('drive/MyDrive/1.DATA/QP/Numeracy600K_headline_test.json', 'r') as file:
    validation_data = json.load(file)
    validation_df = pd.DataFrame(validation_data)

In [ ]:
# Create model dictionary
test_models = {
                'BERT': 'bert-base-uncased', # 0.068
                'RoBERTa': 'roberta-base', # 0.184
                'MathBERT': 'tbs17/MathBERT-custom', # 0.008
                'LinkBERT': 'michiyasunaga/LinkBERT-base', # 0.001
                'FinBERT': 'ProsusAI/finbert', # 0.001
                'ALBERT': 'albert-base-v2', # 0.132
                # 'Xlnet': 'xlnet-base-cased',
               }

def get_model(model_name='BERT',new_mask_word='[Num]'):
  if model_name not in test_models:
    model_name = 'BERT'
  tokenizer = AutoTokenizer.from_pretrained(test_models[model_name])
  model = AutoModelForMaskedLM.from_pretrained(test_models[model_name])

  # Add a new mask word/token
  tokenizer.add_special_tokens({'mask_token': new_mask_word})
  model.resize_token_embeddings(len(tokenizer))
  print(f"Is [Num] a Mask Token: {new_mask_word in tokenizer.get_vocab()}")

  return tokenizer, model

In [ ]:
import random
random_items = random.sample(validation_data, 10)
for item in random_items:
  print(item['masked'], '\t', item['magnitude'], '\t', item['title'])

On the scene: Son Jung Wan Spring [Num] presentation 	 4 	 On the scene: Son Jung Wan Spring 2014 presentation
Oceania Cruises sending R-Class for $[Num] million refurbishment 	 2 	 Oceania Cruises sending R-Class for $50 million refurbishment
Seahawks at or near the top of many Week [Num] Power Rankings 	 1 	 Seahawks at or near the top of many Week 4 Power Rankings
Mega Millions winning numbers drawn: Did anyone win the lottery jackpot May [Num]? 	 2 	 Mega Millions winning numbers drawn: Did anyone win the lottery jackpot May 14?
Girl Scouts have fun at Geocaching [Num] 	 3 	 Girl Scouts have fun at Geocaching 101
Photographing [Num]th fireworks from Cabrillo National Monument 	 1 	 Photographing 4th fireworks from Cabrillo National Monument
Kris Kross Chris Kelly dies at age [Num] 	 2 	 Kris Kross Chris Kelly dies at age 34
ROH news: Main event of 'Final Battle [Num]' announced 	 4 	 ROH news: Main event of 'Final Battle 2013' announced
Indie Music Mondays: August 5th, [Num] - Open

In [ ]:
from sklearn.metrics import f1_score
import math
import statistics

def get_f1m(tokenizer, model, data):
    # Create a pipeline for filling masked tokens using the provided model and tokenizer
    fill_mask = pipeline(
        "fill-mask",
        model=model,
        tokenizer=tokenizer,
    )

    # Initialize lists to store ground truth and predicted magnitudes
    true_magnitudes = []
    predicted_magnitudes = []

    # Calculate predicted magnitudes and collect true magnitudes
    for sample in data[:10]:
        text = sample["masked"]
        ground_truth = sample["magnitude"]  # Ground truth numerical magnitude

        # Tokenize and mask the input text
        tokenized_input = tokenizer(text, return_tensors="pt")
        mask_token_index = torch.where(tokenized_input["input_ids"] == tokenizer.mask_token_id)

        # Model inference for masked token prediction
        predictions = fill_mask(text)
        predicted_magnitudes.append(ground_truth)
        print(predictions[0]['token_str'], 'gt:', ground_truth)

        try:  # Check if the number of predicted tokens is not one
            predicted_token = predictions[0]["token_str"]
        except:
            predicted_tokens = [item['token_str'] for item in predictions[0]]
            predicted_token = statistics.mode(predicted_tokens)

        try:  # Check if the predicted token is a number
            predicted_magnitude = math.floor(math.log10(float(predicted_token)))  # Convert predicted token to float
            try:
                predicted_magnitude += 1
                if predicted_magnitude > 6:
                    predicted_magnitude = 6
                true_magnitudes.append(predicted_magnitude)
            except:
                true_magnitudes.append(0)
        except ValueError as e:
            true_magnitudes.append(0)

    # Calculate the F1 score based on predicted and true magnitudes
    macro_f1 = f1_score(true_magnitudes, predicted_magnitudes, average='macro')
    print(f"Macro F1 score based on predicted magnitudes: {macro_f1}")

In [ ]:
# Iterate through each model name in the 'test_models' list
for model_name in test_models:
    print(model_name, ': ')  # Print the current model name

    # Get the tokenizer and model corresponding to the current model name
    tokenizer, model = get_model(model_name)

    num_groups = 300  # Define the number of groups to create

    random_groups = []  # Initialize an empty list to store randomly split groups of data

    # Create 'num_groups' random groups by splitting the 'validation_data'
    for _ in range(num_groups):
        _, group = train_test_split(validation_data, test_size=0.2, random_state=np.random.randint(100))
        random_groups.append(group)

    # Access each randomly created group for evaluation
    for i, group in enumerate(random_groups):
        # Evaluate the F1 score for the current tokenizer, model, and each random group
        get_f1m(tokenizer, model, random_items)

## Inference QQA

Code Reference: https://github.com/allenai/numglue

In [ ]:
# Load data from google drive
from google.colab import drive
drive.mount('/content/drive')

# Load and preprocess training dataset from the JSON file
with open('drive/MyDrive/1.DATA/QQA/QQA_train.json', 'r') as file:
    training_data = json.load(file)
    training_df = pd.DataFrame(training_data)

# Load and preprocess validation dataset from another JSON file
with open('drive/MyDrive/1.DATA/QQA/QQA_dev.json', 'r') as file:
    validation_data = json.load(file)
    validation_df = pd.DataFrame(validation_data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(training_df.head(1))

                                            question     Option1      Option2  \
0  The ranger and the rustler both were riding ho...  the ranger  the rustler   

     answer    type                                   question_sci_10E  \
0  Option 2  Type_3  The ranger and the rustler both were riding ho...   

                                       question_char  \
0  The ranger and the rustler both were riding ho...   

                               question_sci_10E_char  \
0  The ranger and the rustler both were riding ho...   

                                       question_mask  
0  The ranger and the rustler both were riding ho...  


In [ ]:
for index, row in training_df.iterrows():
    print(row['input'][:50], row['answer'])
    if index == 10:
      break

The ranger and the rustler both were riding horses Option 2
Tina is racing her two dogs. Her greyhound weighs  Option 1
Mike and Sue decide to ride their bikes around the Option 2
A tank weighs around 63 tons. A toy car weighs 1.5 Option 2
The mammoth moved at a speed of 21 km per hour thr Option 2
The mammoth moved at a speed of 18 km per hour thr Option 1
Sarah used a remote control to turn on two identic Option 1
A sedan weighs 1500 Kg and a garbage truck which w Option 1
The beauty queen glided across the marble floors w Option 1
Rolling a marble over dirt creates 1.2 mega N resi Option 1
Marcus's son took the pool ball off the pool table Option 1


In [ ]:
# Create a dictionary containing model names as keys and their respective pre-trained model identifiers as values
test_models = {
    'BERT': 'bert-base-uncased',
    'RoBERTa': 'roberta-base',
    'MathBERT': 'tbs17/MathBERT-custom',
    'LinkBERT': 'michiyasunaga/LinkBERT-base',
    'FinBERT': 'ProsusAI/finbert',
    'ALBERT': 'albert-base-v2',
}

def get_model(model_name='BERT'):
    """
    Retrieves the tokenizer and model based on the specified model name.

    Args:
    - model_name: Name of the pre-defined model (default is 'BERT')

    Returns:
    - tokenizer: AutoTokenizer object initialized with the corresponding pre-trained tokenizer
    - model: AutoModelForMaskedLM object initialized with the corresponding pre-trained model
    """
    # Check if the specified model name exists in the test_models dictionary; default to 'BERT' otherwise
    if model_name not in test_models:
        model_name = 'BERT'

    # Initialize the tokenizer with the pre-trained tokenizer for the specified model
    tokenizer = AutoTokenizer.from_pretrained(test_models[model_name])

    # Initialize the model with the pre-trained model weights for the specified model
    model = AutoModelForMaskedLM.from_pretrained(test_models[model_name])

    return tokenizer, model

In [ ]:
def parse_question(sentence):
    """
    Parses a sentence into segments, separating the question segment from the rest.

    Args:
    - sentence: The input sentence to be parsed

    Returns:
    - text_segment: Textual segment of the sentence (excluding the question)
    - question_segment: Question segment extracted from the sentence
    """
    segments = sentence.split('.')  # Split the sentence into segments using periods as delimiters
    question_segment = segments[-1]  # Extract the last segment as the question segment
    text_segment = ''.join(segments[:-1])  # Combine the preceding segments as the text segment
    return text_segment, question_segment.strip()  # Return the text segment and the stripped question segment

In [ ]:
def select_answer(start_scores, end_scores):
    # Find the maximum start and end positions
    max_start = torch.argmax(start_scores)
    max_end = torch.argmax(end_scores)

    # Ensure the start position is not greater than the end position
    if max_start <= max_end:
        return max_start, max_end
    else:
        # If the start is greater than the end, find the best span within the limits
        best_span = (max_start, max_start)  # Initialize with the start position

        # Find the best span by maximizing the sum of start and end scores
        max_score = start_scores[max_start] + end_scores[max_start]
        for end in range(max_start, min(len(end_scores), max_start + 20)):
            score = start_scores[max_start] + end_scores[end]
            if score > max_score:
                max_score = score
                best_span = (max_start, end)

        return best_span

In [ ]:
def select_answer(start_scores, end_scores):
    """
    Selects the best answer span from given start and end scores.

    Args:
    - start_scores: Tensor containing scores for the start positions
    - end_scores: Tensor containing scores for the end positions

    Returns:
    - best_span: Tuple containing the start and end positions of the selected answer span
    """
    # Find the position with the maximum score for start and end positions
    max_start = torch.argmax(start_scores)
    max_end = torch.argmax(end_scores)

    # Ensure that the start position is not greater than the end position
    if max_start <= max_end:
        return max_start, max_end
    else:
        # If the start is greater than the end, find the best span within the limits
        best_span = (max_start, max_start)  # Initialize the best span with the start position

        # Find the best span by maximizing the sum of start and end scores within a limit of 20 positions
        max_score = start_scores[max_start] + end_scores[max_start]
        for end in range(max_start, min(len(end_scores), max_start + 20)):
            score = start_scores[max_start] + end_scores[end]
            if score > max_score:
                max_score = score
                best_span = (max_start, end)

        return best_span  # Return the tuple containing the best start and end positions

## Classification QQA

In [ ]:
import torch
from transformers import RobertaForQuestionAnswering, RobertaTokenizer
from transformers import Trainer, TrainingArguments

# Example input
context = "The ranger and the rustler both were riding horses that galloped at the same speed. " \
          "The rustler left at 01:00 where as the ranger left at 0500 hours. " \
          "Who has traveled further?? Option 1: the ranger, Option 2: the rustler"

# Example output
expected_output = 'Option 2'

# Tokenize the input
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
inputs = tokenizer(context, return_tensors="pt")

# Create labels (0 for 'Option 1' and 1 for 'Option 2')
label = 1 if expected_output == 'Option 2' else 0

# Fine-tune RoBERTa for question answering and binary classification
model_qa = RobertaForQuestionAnswering.from_pretrained('roberta-base')
model_classifier = torch.nn.Linear(model_qa.config.hidden_size, 2)  # Binary classifier layer

# Training the binary classifier
optimizer = torch.optim.Adam(model_classifier.parameters(), lr=1e-5)

# Fine-tuning loop (This is a simplified version and requires proper data preparation and batching for real training)
for epoch in range(5):  # 5 epochs for demonstration
    optimizer.zero_grad()
    qa_output = model_qa(**inputs)
    logits = model_classifier(qa_output.pooler_output)  # Use QA output for classification
    loss = torch.nn.functional.cross_entropy(logits.view(1, -1), torch.tensor([label]))  # Compute loss
    loss.backward()
    optimizer.step()

# Making predictions
with torch.no_grad():
    qa_output = model_qa(**inputs)
    logits = model_classifier(qa_output.pooler_output)
    predicted_label = torch.argmax(logits).item()

# Mapping predicted label to answer
predicted_output = 'Option 2' if predicted_label == 1 else 'Option 1'
print("Predicted Output:", predicted_output)